# Item Based Filtering(Market)

본 데이터는 포스코 청년 AI/Bigdata 아카데미 과정에서 제공 받은 것으로 국내 한 유기농마트 브랜드의 판매, 고객 정보가 가공된 형태로 제공된 분석용 데이터입니다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ast
from itertools import combinations


In [2]:
data = pd.read_csv('dt_A.csv')
data

,공급일자,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,구매금액,구매매장,반품_원거래일자,구매시각,월,요일
0,2018-01-02,220954944,정상회원,반찬,두부/유부,두부,두부(1모:420g),1.0,3829,매장A,NaN,10:32,1,화
1,2018-01-02,89023040,정상회원,채소,열매채소,단호박/손질,단호박(깍뚝썰기/300g),1.0,4977,매장A,NaN,10:41,1,화
2,2018-01-02,354941440,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,7083,매장A,NaN,11:00,1,화
3,2018-01-02,786634048,탈퇴,반찬,어묵/묵,묵,도토리묵(420g:1모),1.0,7083,매장A,NaN,11:28,1,화
4,2018-01-02,438543104,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,7083,매장A,NaN,11:59,1,화
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52554,2018-10-31,1110819712,정상회원,채소,뿌리채소,무,무(0.7~1kg미만),1.0,2680,매장A,NaN,13:26,10,수
52555,2018-10-31,1517953728,정상회원,채소,양념채소,쪽파,쪽파(500g),1.0,4212,매장A,NaN,10:58,10,수
52556,2018-10-31,713691392,정상회원,반찬,밑반찬,조림,우엉조림(반찬용/150g),1.0,7658,매장A,NaN,14:36,10,수
52557,2018-10-31,247926336,정상회원,간식,빵,건강곡물빵,감자바게트(180g)/매장,1.0,5169,매장A,NaN,16:23,10,수


## 데이터 정제
----------------------------------------------------------  
<b>데이터 정렬</b>
- 고객별 구매 이력, 날짜 순으로 정렬
- 하루를 기준으로 1회 구매했다고 가정해서 어떤 물품을 한 번에 구매했는지 알아볼 수 있음

<b>이상치 제거</b>  
- 구매수량이 음수인 데이터가 존재하는데 반품 이력이므로 제거  

<b>물품명에서 중량(개수) 제거</b>  
- 어떤 물품을 구매했는지 정보만 필요. 중량 정보는 필요 없으므로 중량 정보를 제거한 물품명 정보를 물품명_중량제거 변수에 할당

In [3]:
# 데이터 정렬(회원별, 구매날짜별 기준)
sort_data = data.sort_values(['회원번호','공급일자']).reset_index(drop = True)

# 반품 이력은 제거
sort_data = sort_data[sort_data['구매수량'] > 0].reset_index(drop = True)

# 물품명 기준인데 중량까지 고려할 필요는 없잖아?
sort_data['물품명_중량제거'] = ""
for i in sort_data.index:
    sort_data['물품명_중량제거'][i] = sort_data['물품명'][i].split('(')[0]
sort_data

/home/piai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,공급일자,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,구매금액,구매매장,반품_원거래일자,구매시각,월,요일,물품명_중량제거
0,2018-01-23,18764160,정상회원,생활용품,생활용품,치약/칫솔,물사랑치약(150g*2개),1.0,8806,매장A,NaN,15:41,1,화,물사랑치약
1,2018-03-30,18764160,정상회원,생활용품,생활용품,위생용품,면생리대(팬티라이너/3개),1.0,21059,매장A,NaN,16:55,3,금,면생리대
2,2018-01-11,20235968,정상회원,채소,잎/줄기채소,달래,달래(100g),1.0,4595,매장A,NaN,17:40,1,목,달래
3,2018-01-11,20235968,정상회원,채소,콩나물,콩나물,콩나물(300g),1.0,2489,매장A,NaN,17:40,1,목,콩나물
4,2018-01-22,20235968,정상회원,반찬,두부/유부,두부,찌개용두부(420g),2.0,7275,매장A,NaN,17:17,1,월,찌개용두부
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51488,2018-10-22,1670400000,탈퇴,채소,버섯,표고버섯,말린표고버섯(150g),1.0,19048,매장A,NaN,15:06,10,월,말린표고버섯
51489,2018-10-22,1670400000,탈퇴,건강,건강일반,건강차,건여주(100g),1.0,19718,매장A,NaN,15:06,10,월,건여주
51490,2018-10-27,1670400000,탈퇴,수산,해조,미역/다시마,다시마(200g),1.0,7658,매장A,NaN,12:19,10,토,다시마
51491,2018-10-27,1670400000,탈퇴,채소,버섯,팽이버섯,팽이버섯(150g),1.0,1149,매장A,NaN,12:19,10,토,팽이버섯


In [4]:
print("중량 제거 전 물품명 수 : %d" %len(data['물품명'].unique()))
print("중량 제거 후 : %d" %len(sort_data['물품명_중량제거'].unique()))

중량 제거 전 물품명 수 : 1985
중량 제거 후 : 1569


## 구매 내역 데이터 생성
---------------------
<b>생성 기준</b>
- 하루 기준 동일인물(같은 회원번호)이 구매한 내역을 리스트화
- 한 개의 물품만 구매한 내역은 어떤 물품과 같이 구매했는지 알 수 없으므로 삭제
- 최소 2개 이상의 물품을 구매한 내역으로 데이터 생성

In [9]:
total = [] # 모든 구매 내역 저장 리스트
tmp = []   # 단일 구매 내역 저장 리스트

for i in sort_data.index:
    tmp.append(sort_data['물품명_중량제거'][i])
    if i+1 == len(sort_data): # 마지막 transaction과 이전까지의 구매 내역을 total에 저장
        total.append(tmp)
        break
    # 공급일자나 회원번호가 다른 경우 한 번에 구매한 transaction이 아니므로 
    # 이전까지의 구매 내역 리스트를 total에 저장하고 tmp 초기화
    if (sort_data['공급일자'][i] != sort_data['공급일자'][i+1]) or (sort_data['회원번호'][i] != sort_data['회원번호'][i+1]):
        if len(tmp) >= 2: # 하나만 구매한 transaction은 제외
            total.append(tmp)
            print(tmp)
            tmp = []

['물사랑치약', '면생리대']
['달래', '콩나물']
['찌개용두부', '한우스지도가니']
['대파', '물사랑 프로폴리치약']
['콩나물', '씨암탉떡갈비']
['콩나물', '맛간장', '닭가슴살']
['보리새우', '오이', '바지락살', '채썬오징어']
['풋고추', '구운자른김', '월동무', '한우분쇄육']
['한우국거리', '절편', '오징어채무침', '찹쌀막걸리']
['꼬마소시지', '무시래기', '당근']
['유정란/매장용', '돈가스소스']
['한우분쇄육', '딸기/']
['엿기름가루', '한우산적', '냉동떡국떡', '한우국거리', '월동무']
['구운자른김', '흑임자인절미']
['쌀식빵', '옥수수과자']
['구운자른김', '약식', '한우고기곰국']
['동물복지유정란', '구운자른김']
['말랑말랑백미가래떡', '자연그대로 아이크림']
['약식', '한우분쇄육', '중성세제 샘플']
['한우산적', '두텁찹쌀떡', '우엉']
['콩나물', '당근']
['김밥용햄', '유정란', '시금치']
['한우분쇄육', '우엉']
['찹쌀막걸리', '당근', '한우산적']
['탕종식빵', '한우산적']
['구운자른김', '말랑말랑백미가래떡', '두부']
['한우분쇄육', '양송이스프', '흑임자구름떡']
['흑임자구름떡', '한입쏙쏙젤리']
['무', '당근']
['두부', '완숙토마토', '두부']
['두부', '구죽염가루']
['감자라면', '쑥개떡', '꾸러기짱']
['한우분쇄육', '훈제족발', '쑥개떡']
['완숙토마토', '콩나물', '통닭', '유정란/매장용', '숲사랑 미용사각휴지']
['당근', '우엉', '당근', '무시래기', '우엉']
['방울토마토/대추형', '오덕원찹쌀고추장', '유정란', '들기름']
['유정란', '무']
['우유식빵', '옥수수플레이크', '쑥인절미']
['우리밀흰밀국수', '콩국물']
['오이', '쑥인절미']
['백숙용통닭', '양송이스프', '한우분쇄육']
['유정란', '찌개용앞다리/조각', 

['유정란/매장용', '중파', '오이']
['오이', '생알땅콩']
['파프리카', '숲사랑 미용사각휴지', '깐마늘/난지형/유']
['양파', '곰취', '칠분도미/유']
['느타리버섯', '감귤꽁꽁', '오미자꽁꽁']
['포도/유/캠벨', '콜라비', '깐마늘/난지형/유', '콩국물']
['동치미물냉면', '콩국물', '유정란/매장용']
['백미/유', '상추', '사과']
['못난이사과/유', '귀리', '유자차', '시금치']
['등뼈', '대파']
['오리롤로스', '딸기잼']
['단호박품은쌀과자', '단호박품은쌀과자', '유정란/매장용', '시금치']
['우동면', '감자']
['동물복지유정란', '콩국물', '쌀사랑국수']
['황토숙주나물', '곱게다진대구살']
['양파', '딸기꽁꽁', '포도꽁꽁']
['오이', '카레', '청경채']
['참다래잼', '포도잼']
['훈제족발', '칠분도미/유']
['마스코바도', '두유']
['찰옥수수', '유기농요거트딸기아이스크림']
['소고기미역국', '훈제족발']
['유정란/매장용', '양파', '구운참김', '감자라면']
['유정란/매장용', '한울사과즙', '당근']
['단호박품은쌀과자', '닭다리순살']
['미역줄기', '애플수박']
['얼갈이', '숲사랑 미용사각휴지', '아카시아꿀', '호박잎', '양파']
['유정란/매장용', '콩나물', '두부', '한재미나리']
['양파', '명태참어묵', '시금치']
['얼음식혜', '파김치', '구운자른김', '콩나물', '호박죽', '당근']
['양파', '흑토마토', '동치미냉면육수', '우엉']
['복숭아', '복숭아', '홍삼액']
['깐마늘/난지형/유', '쌈양상추']
['유정란/매장용', '감자/소', '숲사랑 3겹둥근휴지']
['보리새우', '피망', '미역']
['방울토마토/대추형', '사골곰국', '복숭아']
['미숫가루', '닭고기볶음밥', '유정란/매장용', '동치미냉면육수', '조림용감자']
['구운전장김', '꽈리고추', '구운자

['쌈양상추', '깐녹두/무']
['냉동백미깨송편', '자장면', '현미가래떡']
['연두부', '말랑말랑백미가래떡']
['브로컬리', '매운감자라면']
['루꼴라', '양파', '로메인', '참깨샐러드소스']
['말랑말랑백미가래떡', '볶은왕소금', '통밀식빵', '절편']
['찌개용두부', '순두부']
['호박고구마', '냉동모듬찰떡']
['대파', '남해안생굴', '잎마늘']
['로메인', '제주백조기 모듬어묵', '떠먹는요거트', '금귤/유,무', '깐쪽파', '시금치', '떠먹는요거트']
['우유', '달지않은 떠먹는요거트']
['플레인요구르트', '냉동블루베리']
['두부', '냉동모듬찰떡']
['중파', '오이']
['저지방우유', '명태참어묵']
['중파', '돼지모듬구이용']
['꽈리고추', '가지']
['순두부', '참기름']
['가시오이', '순두부']
['플레인요구르트', '양배추', '생물가자미']
['비름나물', '무시래기', '냉동모시송편']
['김부각', '두부']
['콩나물', '유정란/매장용', '구운전장김', '콩비지', '도토리묵', '솔부추', '샐러리', '칠분도미/유']
['솔뫼찹쌀고추장', '마늘/난지형']
['애호박', '뽕나물']
['대파', '차조/국산']
['당근', '통살생선가스']
['등심', '등심']
['찹쌀누룽지', '국물멸치']
['만가닥버섯모음', '찹쌀누룽지']
['구운김밥김', '찹쌀누룽지', '반마른오징어']
['사골곰국', '주먹밥해물', '해물모음']
['두부', '얼린오징어']
['양지국거리용', '들기름', '콩나물', '반마른오징어']
['채도라지', '약과']
['바로먹는카레', '두부', '적양배추', '구운김밥김', '구운자른김', '시금치', '수피아 누리봄 온몸용로션']
['당근', '수피아 누리봄 온몸용로션', '물사랑 주방용물비누']
['오징어동그랑어묵', '두부', '양배추', '당근', '시금치']
['구운김밥김', '하늘바다새우', '오징어동그랑어묵', '적양배추']

['당근', '굵은소금']
['우유', '콩비지']
['부침가루', '상큼한감귤&한라봉', '흰밀가루', '기장/유']
['파래참김', '감귤꽁꽁', '포도꽁꽁', '찰옥수수퐁', '오미자꽁꽁']
['곱게다진한우순살', '피자만두', '감귤꽁꽁']
['생강가루', '훈제오리슬라이스']
['녹차쿠키', '냉동흑미건강떡']
['곱게다진한우순살', '간장소스순살닭강정']
['꼬마소시지', '육수용한우곰국']
['과일푸딩', '동물복지유정란', '볶은알땅콩/무']
['꼬마소시지', '새우살', '도토리묵']
['두부', '파래전장김']
['방울토마토/대추형', '과일푸딩']
['유기농우유', '쇠머리떡']
['딸기/', '유정란/매장용']
['앉은뱅이밀가루', '두부']
['콩나물', '찌개용두부']
['당근', '얼음식혜']
['볶음멸치', '얼음식혜']
['어린잎채소', '두부']
['목초우유', '유정란/매장용', '콜라비', '수피아 알로에베라잎즙 수딩젤']
['냉면사리', '명태참어묵']
['오분도미/유', '찌개용두부']
['매운감자라면', '과일푸딩', '명태참어묵', '블루베리요구르트']
['양파', '찌개용두부', '쥬키니호박']
['볶은피땅콩', '국물멸치/상', '대파']
['두부', '한우샤브샤브', '백미/유']
['딸기/', '유정란/매장용']
['불고기용', '얼갈이']
['참외', '연근채']
['백미/유', '한재미나리', '생콩가루', '재래종중파']
['구운유정란', '열무얼갈이물김치']
['쇠미역', '구운유정란']
['들기름', '두부', '중파']
['국물멸치', '쇠미역', '유정란/매장용', '구운김밥김', '콩국물']
['얇은삼겹살', '열무', '오이']
['누룽지', '소보로단팥빵']
['유정란/매장용', '한우국거리', '깻잎']
['유정란/매장용', '오이', '자연그대로 녹차 트러블로션']
['구운유정란', '명태참어묵', '페퍼민트']
['미니단호박', '한우샤브샤브', '유정란/매장용', '둥근애호박', '수피

['냉동백미깨송편', '냉동단호박녹두송편', '냉동쑥깨송편']
['물사랑치약', '얼음식혜']
['얼음식혜', '튀김옷돈가스', '얼음식혜']
['얼음식혜', '사골곰국']
['당근', '알비트']
['생강편강', '흑미참식']
['시금치', '애호박']
['콩나물', '콜리플라워']
['아카시아꿀', '메론_머스크/소', '포도/무/캠벨']
['우엉채', '연근채']
['통삼겹살', '동물복지유정란', '찌개용두부']
['두부', '흰밀가루', '양배추']
['잎마늘', '가지말림', '아침빵', '대파']
['동물복지유정란', '삶은찰옥수수', '상추', '보리순', '명태참어묵']
['쌈다시마', '시금치']
['유정란/매장용', '황금고구마', '찌개용두부']
['유정란/매장용', '콩나물', '양파']
['구운유정란', '쌈다시마']
['명태참어묵', '메밀차']
['콩나물', '숲사랑 주방휴지']
['흑미', '치즈돈가스']
['군만두', '물만두']
['한우불고기부리토', '과일푸딩']
['등심', '뒷다리살', '분쇄육']
['도토리가루', '물사랑 주방용물비누']
['딸기잼', '딸기잼']
['물사랑 세탁가루비누', '유정란/매장용']
['두부', '면생리대']
['한우분쇄육', '시금치', '김치만두']
['생칡즙', '미역', '생귀리가루']
['감자라면', '두텁찹쌀떡', '카스텔라인절미']
['옥수수차', '유기농성이시돌목장우유']
['김밥용햄', '대파']
['김치만두', '유기농아이스크림', '짜지않은볶음멸치']
['깻잎', '당근']
['우엉조림', '깻잎']
['브로콜리새우죽', '복숭아']
['찌개용두부', '호박즙']
['양상추', '귤/유']
['채도라지', '마요네즈', '떡볶이떡', '팽이버섯', '구운김밥김', '루꼴라']
['찌개용두부', '쪽파']
['딸기/', '생물가자미', '상추', '다시마', '대파']
['크루아상', '쇠머리떡']
['훈제족발', '대파', '팽이버섯']
['대파', '설도다짐육', '과일

...

['두부', '파래전장김', '제주백조기 찐어묵', '불고기양념소시지']
['우동면', '유정란/매장용']
['뒷다리살', '유정란/매장용']
['상큼한감귤&한라봉', '불고기양념소시지']
['당근', '구운김밥김', '두부', '닭고기볶음밥']
['유기농아이스크림', '아이스바우유']
['현미유', '불고기양념소시지']
['두부', '상큼한감귤', '콩나물']
['콩나물', '과일푸딩']
['카레', '쫄면']
['감귤즙', '아침빵', '콩나물']
['파래전장김', '유정란/매장용', '순두부', '당근', '브로컬리']
['오이', '두부']
['상큼한포도', '둥근긴어묵', '매운감자라면', '자장소스']
['두부', '상큼한감귤']
['유정란/매장용', '소불고기볶음밥', '콩비지']
['시금치', '동치미물냉면']
['오이', '유정란/매장용', '감자라면']
['닭가슴살', '유기농블루베리아이스바']
['밥풀과자', '콩비지', '냉면사리', '닭가슴살']
['냉면사리', '후랑크소시지', '콩나물']
['두부', '유정란/매장용', '콩나물']
['7곡딸기롤과자', '유기농토마토케찹', '분쇄육', '두부', '바로먹는카레']
['참깨샐러드소스', '양상추']
['양배추', '딸기꽁꽁']
['우리밀빵가루', '동물복지유정란/중란', '구운전장김']
['둥근긴어묵', '구운전장김', '유정란/매장용']
['유정란/매장용', '감자', '등심', '오이']
['사과', '콩나물', '둥근긴어묵', '유정란/매장용']
['두부', '당근']
['찹쌀현미/유', '현미유']
['시금치', '봄동']
['구이용삼치', '사과/']
['참다래', '양파']
['냉이', '포도즙']
['유정란/매장용', '대파', '포도즙']
['배즙', '포도즙', '현미유']
['칼라방울토마토', '오미자꽁꽁', '등뼈']
['구이용삼치', '오미자꽁꽁']
['구이용삼치', '상큼한포도', '고구마부리토']
['오미자꽁꽁', '딸기꽁꽁', '포도꽁꽁', '양파', '블루

['딸기머핀', '유정란/매장용']
['콩나물', '두부']
['꿀선물모음실속형', '표고맛국물팩', '차례상약과']
['돈가스소스', '구운도시락김']
['당근', '우유또띠아']
['찌개용두부', '부숴먹는라면']
['유기쌀올리고당', '카스텔라']
['콩나물', '부침가루']
['찌개용두부', '우유또띠아']
['찌개용두부', '유기쌀올리고당']
['딸기잼', '단무지']
['양파', '쌈다시마']
['청경채', '치커리']
['콩나물', '양파']
['찌개용두부', '연두부']
['미역줄기', '파프리카']
['목이버섯', '알토란', '고구마묵', '팽이버섯', '연근채', '막장']
['팽이버섯', '양파', '양배추', '청포묵', '찌개용두부']
['현미유', '미르25', '현미유']
['채소쌀과자', '백미뻥과자']
['당근', '물사랑 어린이치약']
['잎마늘', '숙주나물']
['중파', '꼬마소시지', '만가닥버섯모음', '팽이버섯', '여행용화장지']
['유기농블루베리아이스바', '오미자꽁꽁']
['백미/유', '완숙토마토', '팽이버섯']
['참맛가루/혼합조미료', '포도잼']
['찌개용두부', '7곡딸기롤과자', '금산오미자원액', '동물복지유정란']
['양파', '옥수수병조림', '볶은알땅콩/무']
['백미/유', '채썬갑오징어', '한우국거리/조각', '구이갈비', '앞다리불고기용']
['풋귤', '우유식빵']
['가시복분자즙', '허브혼합차']
['양송이버섯', '당근', '얇은삼겹살', '채소쌀과자', '옥수수과자']
['선물상자 1호', '자연그대로 페이스동백오일', '수피아 연 영양크림']
['애호박', '우유식빵']
['꼬마새송이버섯', '마스코바도']
['우리콩된장', '국물멸치', '깻잎', '동물복지유정란']
['두부', '옥수수차', '두유']
['바지락살', '탕종식빵', '양송이버섯']
['옥수수차', '찌개용두부', '구운도시락김']
['유정란/매장용', '프로폴리스']
['백미/유', '새우살', '구운김

['쌍화차', '하늘땅감자']
['섬유용탈취제', '채소볶음밥']
['브로콜리새우죽', '소고기채소죽', '물사랑치약']
['청포도/유', '쌍화차']
['전복죽', '고추장멸치볶음']
['소불고기볶음밥', '팽이버섯']
['순두부', '단팥빵']
['두부', '브로컬리', '쌀로만든잉글리쉬머핀', '둥근긴어묵', '현미/유', '목심살']
['죽염', '방울토마토/대추형']
['팽이버섯', '구운참김', '얼음식혜']
['쥬키니호박', '갈치']
['곱게다진한우순살', '순두부']
['분쇄육', '팥죽', '우유']
['사과/', '수정과', '호박죽', '양송이버섯']
['쑥', '유기농순한발아보리차', '초고추장']
['콩비지', '청포묵']
['두부', '양배추']
['곤드레나물밥', '흑미블루베리크림빵']
['크림빵', '고기만두']
['깻잎', '우엉채', '유정란/매장용', '쥬키니호박', '팽이버섯']
['포기로메인', '쫄면']
['양배추', '소보로빵', '오이']
['자장가루', '참외', '스파게티', '소고기스프']
['백미/유', '고기만두']
['쌈양상추', '자연그대로 베이비썬크림', '돼지고기동그랑땡']
['참외', '막국수']
['감귤/청로', '장어탕', '소고기스프']
['콩비지', '팽이버섯']
['연두부', '동물복지유정란', '차례상약과']
['팽이버섯', '백미/유', '막국수', '황태채']
['팽이버섯', '단호박품은쌀과자', '막국수', '쌈장']
['유정란/매장용', '수피아 유아로션', '갈색양송이버섯', '물사랑 주방용물비누']
['떡국떡', '크림스프']
['순살광어가스', '디포리', '막국수', '자장면']
['녹두전', '찹쌀막걸리']
['우유', '유정란/매장용', '크림스파게티', '찌개용두부']
['백미/유', '소보로단팥빵']
['수피아 누리봄 온몸용워시', '순살광어가스', '스파게티', '흑마늘액', '김말이', '우유식빵']
['유정란/매장용', '스파게티소스', '크림스파게티소스', '

['사과당근즙', '딸기잼', '체다치즈', '블루베리요구르트']
['동그랑쌀과자', '두부']
['해물맛국물팩', '모과차', '마늘장아찌']
['유정란', '국물멸치']
['여행용화장지', '상큼한감귤&채소', '양송이버섯']
['구운도시락김', '상큼한포도', '상큼한감귤']
['자연산골뱅이', '호박품은팥시루떡']
['콩나물', '시금치']
['매운맛돈육포', '오리롤로스', '통연근', '제주감자/못난이']
['볶은참깨', '양파']
['두부과자', '단호박품은쌀과자', '불고기갈비양념']
['두유', '구운도시락김', '백미뻥과자']
['딸기머핀', '구운도시락김']
['구운도시락김', '7곡딸기롤과자', '우리밀곡물와플']
['검은콩두유', '달지않은옥수수플레이크']
['양상추', '유정란/매장용', '감자']
['분쇄육', '혼합8곡']
['양상추', '유정란/매장용']
['도토리묵', '우리콩낫토', '새우살', '양파', '연두부', '앞다리찌개용']
['유정란/매장용', '명태참어묵', '쥬키니호박', '진간장', '미역줄기']
['설도다짐육', '양파', '당근']
['연두부', '통밀식빵']
['도토리묵', '당근', '소불고기볶음밥']
['요구르트샐러드소스', '유정란/매장용']
['완숙토마토', '유정란/매장용']
['보리새우', '완숙토마토']
['브로컬리', '사과/']
['요구르트샐러드소스', '오이']
['깐마늘/난지형/유', '연두부']
['칼라방울토마토', '튀김옷돈가스']
['메론_머스크/중', '연두부']
['연두부', '물만두']
['콩나물', '오리롤로스']
['청포도/유', '황기/4년근']
['통밀식빵', '한우불고기꼬마피자']
['고구마부리토', '포도꽁꽁']
['김치만두', '표고맛국물팩', '계란탕솥']
['매일견과', '참기름', '미니파프리카', '중파']
['유정란/매장용', '윈너소시지', '스파게티']
['우유', '고구마양갱', '약과']
['모듬플레이크', '우유']
['숲사랑 3겹둥근휴지', 

['새송이버섯', '우유', '꼬마소시지']
['콩나물', '팽이버섯']
['양파', '채소품은쌀과자', '단호박품은쌀과자']
['우리밀우동', '생활살균수']
['순살삼치', '주먹밥해물']
['소불고기볶음밥', '절편']
['채소볶음밥', '상큼한감귤&한라봉']
['오이피클', '캐모마일']
['쌀식빵', '쌀식빵']
['세탁조 세정제', '돈육포', '만가닥버섯모음']
['산딸기', '구운도시락김', '쌀식빵']
['유정란/매장용', '우유', '생채', '7곡딸기롤과자']
['두부', '유기농감잎차', '산딸기']
['시래기된장국', '한입쏙쏙젤리']
['산딸기', '두부', '찹쌀현미/유']
['사과당근즙', '오징어땅콩볼']
['녹두전', '유기농우유', '깻잎']
['우유식빵', '쌀로만든잉글리쉬머핀', '긁지말고 모물린']
['두부', '설도불고기용', '한우사골국']
['소고기미역국', '항정살']
['딸기잼', '우엉조림', '딸기아침', '우리밀곡물와플']
['냉동블루베리', '돈육포']
['두부', '소고기미역국']
['쑥인절미', '갈색만가닥버섯']
['한우사골국', '유기농성이시돌목장우유']
['구운도시락김', '구운도시락김']
['찹쌀백미/유', '유기농우유', '곤드레나물밥']
['구운유정란', '항정살']
['유기농순한발아보리차', '잔멸치', '김부각']
['구운도시락김', '여행용화장지']
['말린홍합', '유기쌀올리고당', '청포묵']
['참맛핫도그', '백미/유', '하늘바다새우']
['브로콜리새우죽', '상큼한감귤', '참깨샐러드소스', '채소볶음밥']
['우유식빵', '소고기미역국', '딸기잼', '딸기', '유기농저지방우유']
['주먹밥해물', '한입쏙쏙젤리', '한우사골국', '소고기미역국', '전복죽']
['도깨비방망이과자', '포도/무/캠벨']
['유정란/매장용', '유정란/매장용']
['얼음식혜', '유기농성이시돌목장우유', '동태', '치즈돈가스']
['주먹밥채소&해물', '구운김밥김', '쌀리카토니 파스타'

#### 51493 건의 구매 내역 중 2개 이상의 구매가 이루어진 구매 내역은 총 17288건

In [10]:
len(total)

17288

# 추천 알고리즘

### 물품명 별 인덱스 생성
------------------
- 총 1569개의 물품에 고유 인덱스 부여
- 고유 인덱스를 저장한 items 데이터프레임의 feature name을 item_index로 지정

In [11]:
items = sort_data['물품명_중량제거'].drop_duplicates().reset_index()
items['index'] = range(len(items))
items.set_index('물품명_중량제거', inplace = True)
items.rename(columns = {'index' : 'item_index'}, inplace = True)
items

,item_index
물품명_중량제거,
물사랑치약,0
면생리대,1
달래,2
콩나물,3
찌개용두부,4
...,...
사골꼬리곰국,1564
순살삼치,1565
잡곡선물모음,1566


## 아이템 별 동시 구매 건수 Matrix 생성
--------------
- 동시 구매시 +1
- 1회 구매 이력에서 가능한 모든 2개 물품 조합을 추출

In [12]:
result_mat = np.zeros((len(items),len(items)))
result_mat.shape

(1569, 1569)

In [13]:
from itertools import combinations

for i in total:
    ls = list(combinations(i, 2)) # 가능한 모든 2개 물품 조합 추출
    for j in ls:
        index1, index2 = [items['item_index'][j[0]], items['item_index'][j[1]]]
        if index1 != index2: # 같은 물품을 산 경우는 제외
            result_mat[index1, index2] += 1
            result_mat[index2, index1] += 1
result_mat

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 추천 알고리즘 구현
- 입력 물품과 가장 많이 같이 팔렸던 물품을 우선으로 추천


In [14]:
def get_recommend_organic_items(item, items = items, result_matrix = result_mat):
    try:
        idx = items.loc[item]['item_index']
        recommend_item_index = result_matrix[idx].argsort()[::-1][:30]
        recommend_item_values = np.sort(result_matrix[idx])[::-1][:30]
        item_names = pd.Series(recommend_item_index).apply(lambda x : items[items['item_index'] == x].index.values[0])
        result_df = pd.Series(recommend_item_values, index = item_names)
        return result_df
    except:
        print("물품이 존재하지 않습니다.")
        return 0


In [15]:
get_recommend_organic_items('연두부')

두부         17.0
유정란/매장용    16.0
콩나물        12.0
찌개용두부      11.0
구운도시락김      9.0
애호박         9.0
명태참어묵       7.0
감자          7.0
양파          7.0
꼬마찹쌀모나카     7.0
어린잎채소       5.0
유기농우유       4.0
새우살         4.0
동물복지유정란     4.0
순두부         4.0
팽이버섯        4.0
건여주         4.0
대파          4.0
가지          3.0
차례상약과       3.0
쌈양상추        3.0
깻잎          3.0
파래전장김       3.0
부추          3.0
진간장         3.0
도토리묵        3.0
유기보리차       3.0
탕종식빵        3.0
오징어땅콩볼      3.0
사과/         3.0
dtype: float64